# MultiBand Single Pass (MBSP) Demo
This notebook demonstrates a simple implementation of the MBSP methane retrieval algorithm described in Varon et al. (2021).It lets you load a stack of Sentinel-2 images for a point of interest, process each image, and review the results.

## SetupImport the required packages and authenticate Earth Engine.

In [ ]:
import ee
import numpy as np
import geemap
import matplotlib.pyplot as plt

ee.Initialize()

## Utility functions

In [ ]:
def load_sentinel2_stack(lat, lon, start_date, end_date, cloud=20, region_km=2):
    """Return an Earth Engine ImageCollection clipped to a small region."""
    point = ee.Geometry.Point(lon, lat)
    roi = point.buffer(region_km * 500).bounds()
    col = (
        ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(point)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud))
        .map(lambda img: img.clip(roi))
    )
    return col.sort('system:time_start')

def mbsp(b11, b12):
    """Compute MBSP fractional absorption for two numpy arrays."""
    r11 = b11.astype(float)
    r12 = b12.astype(float)
    c = np.sum(r11 * r12) / np.sum(r11 ** 2)
    return np.divide(c * r12 - r11, r11, out=np.full_like(r11, np.nan), where=r11 != 0)

def process_stack(collection):
    """Iterate through the collection and compute MBSP for each image."""
    size = collection.size().getInfo()
    results = []
    for i in range(size):
        img = ee.Image(collection.toList(size).get(i))
        t = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        b11 = geemap.ee_to_numpy(img.select('B11'))
        b12 = geemap.ee_to_numpy(img.select('B12'))
        mbsp_arr = mbsp(b11, b12)
        results.append({'date': t, 'mbsp': mbsp_arr})
    return results

def show_result(result):
    plt.figure(figsize=(6, 6))
    plt.imshow(result['mbsp'], cmap='RdBu')
    plt.title(result['date'])
    plt.colorbar(label='Fractional change')
    plt.show()

## Example usageSpecify the point of interest and time range.

In [ ]:
lat = 31.6585
lon = 5.9053
start = '2020-07-01'
end = '2020-07-31'

collection = load_sentinel2_stack(lat, lon, start, end)
results = process_stack(collection)

for r in results:
    show_result(r)